# Canonical Whitebox Misclassification Attacks Example

## 1. Setup and Imports

This cell imports all the necessary modules from the `adversarial_lab` library, as well as other common libraries like TensorFlow, NumPy, and PIL for image manipulation and numerical operations.

Key `adversarial_lab` modules used:
* `PGD`: Projected Gradient Descent optimizer used to generate adversarial noise.
* `CategoricalCrossEntropy`: Loss function to guide the optimization.
* `WhiteBoxMisclassification`: The main attacker class.
* `AdditiveNoiseGenerator`: Generates the initial noise (can be zeros, random, etc.).
* `PreprocessingFromFunction`: Wraps a custom preprocessing function.
* `POClip`, `PONoisedSampleBounding`: Constraints applied to the noise or the noisy sample.
* `Plotting`: Utility for visualizing images and noise.

In [ ]:
from adversarial_lab.core.optimizers import PGD
from adversarial_lab.core.losses import CategoricalCrossEntropy
from adversarial_lab.attacker.whitebox import WhiteBoxMisclassification
from adversarial_lab.core.noise_generators import AdditiveNoiseGenerator
from adversarial_lab.core.preprocessing import PreprocessingFromFunction
from adversarial_lab.core.constraints import POClip, PONoisedSampleBounding
from adversarial_lab.callbacks import EarlyStopping
from adversarial_lab.arsenal.whitebox import *

from adversarial_lab.utils import Plotting

from PIL import Image
import matplotlib.pyplot as plt

import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image

## 2. Model Selection and Loading

Here, you can choose which pre-trained model to attack. The supported models are `InceptionV3`, `ResNet50`, and `MobileNetV2`.
The subsequent code cell will load the chosen Keras application model with pre-trained ImageNet weights and set the appropriate `input_shape` and model-specific `preprocess_input` and `decode_predictions` functions.


In [ ]:
MODEL = "InceptionV3"       # Supported models: InceptionV3, ResNet50, MobileNetV2

In [ ]:
if MODEL == "InceptionV3":
    from tensorflow.keras.applications import InceptionV3
    from tensorflow.keras.applications.inception_v3 import preprocess_input, decode_predictions
    model = InceptionV3(weights='imagenet')
    input_shape = (299, 299, 3)
elif MODEL == "ResNet50":
    from tensorflow.keras.applications import ResNet50
    from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
    model = ResNet50(weights='imagenet')
    input_shape = (224, 224, 3)
elif MODEL == "MobileNetV2":
    from tensorflow.keras.applications import MobileNetV2
    from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions
    model = MobileNetV2(weights='imagenet')
    input_shape = (224, 224, 3)

## 3. Image Preprocessing Function

Neural network models expect input data in a specific format. This `preprocess` function handles:
* **Type Casting**: Converts the input image data to `float32`.
* **Grayscale to RGB**: Converts grayscale images (2D or 3D with 1 channel) to 3-channel RGB format, as expected by the pre-trained models.
* **Resizing**: Resizes the image to the model's required input dimensions (e.g., (299, 299) for InceptionV3).
* **Batch Dimension**: Adds a batch dimension to the image tensor, as models typically expect batches of images.
* **Model-Specific Preprocessing**: Applies the specific preprocessing steps required by the chosen Keras model (e.g., scaling pixel values to a certain range like [-1, 1] or [0, 1], or BGR conversion if necessary).

In [ ]:
def preprocess(sample, *args, **kwargs):
    input_sample = tf.cast(sample, dtype=tf.float32)
    if len(input_sample.shape) == 2:
        input_sample = tf.expand_dims(input_sample, axis=-1)
        input_sample = tf.image.grayscale_to_rgb(input_sample)

    elif len(input_sample.shape) == 3 and input_sample.shape[-1] == 1:
        input_sample = tf.image.grayscale_to_rgb(input_sample)

    input_tensor = tf.convert_to_tensor(input_sample, dtype=tf.float32)
    resized_image = tf.image.resize(input_tensor, input_shape[:2])
    batch_image = tf.expand_dims(resized_image, axis=0)
    return preprocess_input(batch_image)

## 4. Load and Classify Original Image

This cell loads an example image (`panda.jpg`). We then preprocess it and get the model's prediction on this original image. This establishes a baseline classification before applying any adversarial attack. The top predicted classes and their probabilities are displayed.


In [ ]:
image = Image.open('data/panda.jpg')
image_array = np.array(image)

predictions = model.predict(preprocess(image_array), verbose=0)
print("Predicted class:", decode_predictions(predictions, top=1)[0][0][1])
print("Predicted class index:", np.argmax(predictions, axis=1)[0])
print("Predicted class probability:", np.max(predictions, axis=1)[0])

## FAST SIGN GRADIENT METHOD

In [ ]:
EPSILON = 0.05

attacker = FastSignGradientMethodAttack(
    model=model,
    preprocessing_fn=PreprocessingFromFunction.create(preprocess),
    epsilon=EPSILON,
    binary=False
)

noise, noise_meta = attacker.attack(
    sample=image_array,
    target_class=924,
    on_original=True
)

Plotting.plot_images_and_noise(image_array, noise)

## Projected Gradient Descent

In [ ]:
EPSILON = 0.05
LEARNING_RATE = 1
EPOCHS = 10

early_stopping = EarlyStopping(trigger="misclassification", target_class=924, confidence=0.5)

attacker = ProjectedGradientDescentAttack(
    model=model,
    preprocessing_fn=PreprocessingFromFunction.create(preprocess),
    epsilon=EPSILON,
    binary=False,
    callbacks=[early_stopping],
)

noise, noise_meta = attacker.attack(
    sample=image_array,
    target_class=924,
    on_original=True
)

Plotting.plot_images_and_noise(image_array, noise)

## Carlini Wagner Attack

In [ ]:
C = 15
KAPPA = 0.5
LEARNING_RATE = 1
EPOCHS = 10

early_stopping = EarlyStopping(trigger="misclassification", target_class=924, confidence=0.5)

attacker = CarliniWagnerAttack(
    model=model,
    preprocessing_fn=PreprocessingFromFunction.create(preprocess),
    C=C,
    kappa=KAPPA,
    learning_rate=float(LEARNING_RATE),
    callbacks=[early_stopping],
)

noise, noise_meta = attacker.attack(
    sample=image_array,
    target_class=924,
    on_original=True
)

Plotting.plot_images_and_noise(image_array, noise)

## Deep Fool

In [ ]:
EPSILON = 0.05
EPOCHS = 10
OVERSHOOT = 0.1

early_stopping = EarlyStopping(trigger="misclassification", confidence=0.2)
preprocessing_fn = PreprocessingFromFunction.create(preprocess)

attacker = DeepFoolAttack(
    model=model,
    preprocessing_fn=preprocessing_fn,
    epsilon=EPSILON,
    overshoot=OVERSHOOT,
    callbacks=[early_stopping],
    efficient_mode=10,
)

noise, noise_meta = attacker.attack(
    sample=image_array,
    target_class=924,
)

Plotting.plot_images_and_noise(preprocessing_fn.preprocess(image_array).numpy(), noise)

## Smooth fool

In [ ]:
EPSILON = 0.05
EPOCHS = 10
OVERSHOOT = 0.1
SIGMA = 0.5
KERNEL_SIZE = 5

early_stopping = EarlyStopping(trigger="misclassification", confidence=0.2)
preprocessing_fn = PreprocessingFromFunction.create(preprocess)

attacker = SmoothFoolAttack(
    model=model,
    preprocessing_fn=PreprocessingFromFunction.create(preprocess),
    epsilon=EPSILON,
    overshoot=OVERSHOOT,
    sigma=SIGMA,
    kernel_size=KERNEL_SIZE,
    callbacks=[early_stopping],
    efficient_mode=10,
)

noise, noise_meta = attacker.attack(
    sample=image_array,
    target_class=924,
)

Plotting.plot_images_and_noise(preprocessing_fn.preprocess(image_array).numpy(), noise)